# 02 无头浏览器
有的时候，我们爬虫所用到的网页出于各种因素会有复杂的渲染和组成部分，而这些都不是我们所需要的。所以我们希望能够让浏览器在后端默默地运行，不断返回精简有价值的信息。这就要使用到无头浏览器了。在无头浏览器运行的时候，是看不见网页的。

接下来演示如何处理下拉列表选项框，顺便讲解无头浏览器。

任务：获取[艺恩](https://www.endata.com.cn/BoxOffice/BO/Year/index.html)主界面下拉菜单中的所有年份的电影票房。

如果希望构建无头浏览器，可以引入chrome的配置模块，对浏览器进行配置。

In [1]:
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select#处理select专用的模块。
import time
from selenium.webdriver.chrome.options import Options

In [2]:
# 准备好参数配置
opt=Options()
opt.add_argument('--headless')#设置浏览器为无头模式
opt.add_argument('--disable-gpu')#设置浏览器不显示，不使用gpu。
web=Chrome(options=opt)

下面是任何一个下拉列表的基本格式：

```
<select>
    <option value=1>2023</option>
    <option value=2>2022</option>
    <option value=3>2021</option>
</select>
```

介绍一下，`select`中切换选项卡的方法有三个。
- `select_by_index(m)`：根据索引来切换选项卡。就是第m个option。
- `select_by_value(m)`：根据option的value属性来切换选项。就是切换到value=m的option，以此类推。
- `select_by_visible_text('2023')`：根据option中显示的文本来切换选项，切换到显示2023的那个option。

In [12]:
web.get('https://www.endata.com.cn/BoxOffice/BO/Year/index.html')
sel_elem=web.find_element(By.XPATH,'//*[@id="OptionDate"]')
#这是用通用手段获取网页对象，下面还需要把这个元素封装成一个可迭代的下拉列表对象。
sel=Select(sel_elem)#这样包装之后，就能够让网页自动地访问每一个选项，并且观察以此带来的网页的变化。
#让浏览器自动调整选项
with open('output.txt','w',encoding='utf-8')as f:
    for i in range (len(sel.options)):#i就是每一个下拉框选项的索引位置
        print(f'generating page: {i}.')
        sel.select_by_index(i)
        time.sleep(2)#这个网页的select是ajax的，每次切换都重新发起请求并且渲染。
        table=web.find_element(By.XPATH,'//*[@id="TableList"]/table')
        print(table.text,file=f,end='\n')#打印所有文本信息
        print('======================================================',file=f)
print('done!')
web.quit()

generating page: 0.
generating page: 1.
generating page: 2.
generating page: 3.
generating page: 4.
generating page: 5.
generating page: 6.
generating page: 7.
generating page: 8.
generating page: 9.
generating page: 10.
generating page: 11.
generating page: 12.
generating page: 13.
generating page: 14.
generating page: 15.
done!


**另外，selenium也可以直接拿到完全渲染完毕的页面源代码(不论是否是ajax)。**

In [3]:
web.get('https://www.endata.com.cn/BoxOffice/BO/Year/index.html')
print(web.page_source)

<html><head>
    <title>艺恩-数据智能服务商_年度票房</title>

    <meta charset="utf-8">
    <meta name="referrer" content="no-referrer">
    <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">
    <meta name="viewport" content="width=device-width,initial-scale=1,maximum-scale=1,minimum-scale=1,user-scalable=no">
    <meta name="format-detection" content="telephone=no,email=no">
    <meta name="apple-mobile-web-app-status-bar-style" content="black">
    <meta name="keywords" content="艺恩,endata,艺恩数据,艺恩咨询,票房,电影票房,电影排片,票房统计,排片统计,票房数据库,电影票房数据库,中国票房,北美票房,内地票房">

    <link rel="stylesheet" type="text/css" href="/library/swiper/swiper.min.css">
    <link rel="stylesheet" type="text/css" href="/library/layui/v2.5.5/css/layui.css">
    
    <link rel="stylesheet" type="text/css" href="/css/pages.css?v=2021.10.2">


    <script src="https://hm.baidu.com/hm.js?aeac9d3b78039bf038f68763a5d056fd"></script><script type="text/javascript" src="/library/layui/v2.5.5/layui.all.js"></script><link id="layuic